In [1]:
from tensorflow import keras
import os
import pandas as pd
import cv2
import numpy as np

In [2]:
import tensorflow as tf

In [3]:
print(tf.__version__)

2.17.0


In [4]:
model = keras.models.load_model('./MobileNetV2_binary.keras')

/opt/anaconda3/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 322 variables whereas the saved optimizer has 326 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [5]:
model

<Functional name=functional, built=True>

In [8]:
import os
import cv2
import numpy as np
# 이미지 경로 설정
dlist = './dataset/horse-or-human/test/'
predict_list = [f for f in os.listdir(dlist) if f.lower().endswith(('jpg', 'jpeg', 'png', 'webp'))]  # 이미지 파일만 선택

# 실제 레이블 설정 (파일 이름에 따라 설정)
true_labels = []
for file_name in predict_list:
    if "human" in file_name or "사람" in file_name:  # 사람을 나타내는 파일명에 따라 레이블 설정
        true_labels.append(1)
    elif "horse" in file_name or "말" in file_name:  # 말을 나타내는 파일명에 따라 레이블 설정
        true_labels.append(0)

print(len(true_labels), len(predict_list))

# 실제 레이블과 예측할 이미지 개수가 일치하는지 확인
assert len(true_labels) == len(predict_list), "레이블과 이미지 파일 개수가 일치하지 않습니다."

# 이미지 전처리
img = [cv2.imread(dlist + i) for i in predict_list]
img = [cv2.resize(i, (224, 224)) for i in img]  # 모델 입력 크기에 맞게 이미지 크기 조정
img = np.array(img)
img = img.astype('float32') / 255.0  # 정규화

# 분류
cutoff = 0.5
predictions = model.predict(img)

# 틀린 예측에 대한 정보 저장 리스트
wrong_predictions = []

# 예측 결과 출력 및 맞춘 비율 계산
correct_count = 0
total_count = len(predictions)

for idx, prediction in enumerate(predictions):
    file_name = predict_list[idx]
    confidence = float(prediction[0])  # 예측 확률 (맞춤 비율)
    predicted_label = 1 if confidence >= cutoff else 0  # cutoff에 따른 예측 레이블
    
    # 실제 레이블과 예측 레이블을 비교
    if predicted_label == true_labels[idx]:
        correct_count += 1  # 맞춘 경우 카운트 증가
    else:
        # 틀린 경우의 정보 저장
        wrong_predictions.append((file_name, confidence, predicted_label, true_labels[idx]))
    
    # 파일 이름과 확률 출력
    if predicted_label == 1:
        print(f"{file_name}: 사람, 확률: {confidence:.2f}")
    else:
        print(f"{file_name}: 말, 확률: {1 - confidence:.2f}")

# 전체 맞춘 비율 (정확도) 계산
accuracy = correct_count / total_count
print(f"\n전체 맞춘 비율(정확도): {accuracy * 100:.2f}%")

# 틀린 예측 정보 출력
print("\n틀린 예측 목록:")
for file_name, confidence, predicted_label, true_label in wrong_predictions:
    predicted_class = "사람" if predicted_label == 1 else "말"
    true_class = "사람" if true_label == 1 else "말"
    print(f"{file_name}: 예측 - {predicted_class}, 실제 - {true_class}, 확률: {confidence:.2f}")


235 235
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 469ms/step
human_081.jpeg: 사람, 확률: 0.83
human_51.jpg: 사람, 확률: 1.00
human_023.jpg: 사람, 확률: 0.92
human_28.jpeg: 사람, 확률: 0.99
horse_077.jpeg: 말, 확률: 0.96
horse_098.jpeg: 말, 확률: 0.99
horse_001.jpg: 말, 확률: 0.99
human_078.jpeg: 사람, 확률: 0.87
human_097.jpeg: 사람, 확률: 0.92
human_45.jpeg: 사람, 확률: 0.96
horse_014.jpg: 사람, 확률: 0.96
horse_26.jpeg: 말, 확률: 1.00
human_022.jpg: 사람, 확률: 0.97
human_008.jpg: 사람, 확률: 0.96
human_020.jpg: 사람, 확률: 0.58
human_52.jpg: 사람, 확률: 0.87
human_73.jpeg: 사람, 확률: 0.74
horse_094.jpeg: 말, 확률: 0.99
horse_082.jpeg: 말, 확률: 0.83
horse_002.jpg: 말, 확률: 0.97
human_32.jpeg: 사람, 확률: 0.95
horse_15.webp: 말, 확률: 1.00
human_49.jpeg: 사람, 확률: 0.98
horse_003.jpg: 말, 확률: 0.99
horse_017.jpg: 말, 확률: 0.74
horse_016.jpeg: 말, 확률: 0.96
human_53.jpg: 사람, 확률: 0.84
human_021.jpg: 사람, 확률: 0.95
human_009.jpg: 사람, 확률: 0.85
horse_057.jpeg: 말, 확률: 0.79
human_025.jpg: 사람, 확률: 0.99
human_019.jpg: 사람, 확률: 0.95
horse_013.jpg: 말, 확률: 0.87
horse_007.jpg: 말, 확률: 0.90
human_48.